In [187]:
import pandas as pd
import numpy as np
import datetime

structure = pd.read_csv('course-217-structure.csv')
events = pd.read_csv('course-217-events.csv')
events['time_f'] = pd.to_datetime(events['time'], unit='s')

In [188]:
print(structure.head(), '\n')
print(structure.dtypes)

   course_id  module_id  module_position  lesson_id  lesson_position  step_id  \
0        217        614                2      13228                2    38842   
1        217        614                2      13228                2    39715   
2        217        614                2      13228                2    39716   
3        217        614                2      13228                2    39717   
4        217        614                2      13228                2    39721   

   step_position    step_type  step_cost  
0              1         text          0  
1              6         code          1  
2              7         code          1  
3              8         code          1  
4              9  free-answer          0   

course_id           int64
module_id           int64
module_position     int64
lesson_id           int64
lesson_position     int64
step_id             int64
step_position       int64
step_type          object
step_cost           int64
dtype: object


In [189]:
print(events.head(), '\n')
print(events.dtypes)

   user_id  action  step_id        time              time_f
0        1  viewed    47435  1465181036 2016-06-06 02:43:56
1        1  viewed    47438  1465181034 2016-06-06 02:43:54
2        1  viewed    47435  1465181021 2016-06-06 02:43:41
3        1  viewed    47436  1465181015 2016-06-06 02:43:35
4        1  viewed    41605  1465180037 2016-06-06 02:27:17 

user_id             int64
action             object
step_id             int64
time                int64
time_f     datetime64[ns]
dtype: object


In [190]:
events = pd.merge(
    left = events,
    right = structure[['step_id', 'step_cost']], 
    how = 'left', 
    on = 'step_id')

print(events.head())

   user_id  action  step_id        time              time_f  step_cost
0        1  viewed    47435  1465181036 2016-06-06 02:43:56          0
1        1  viewed    47438  1465181034 2016-06-06 02:43:54          0
2        1  viewed    47435  1465181021 2016-06-06 02:43:41          0
3        1  viewed    47436  1465181015 2016-06-06 02:43:35          0
4        1  viewed    41605  1465180037 2016-06-06 02:27:17          1


In [191]:
# отсортируем таблицу по time
events.sort_values('time', inplace = True)

# выберем уникальные user_id и создадим словарь для подсчёта затраченого времени
unique_user_id = set(events['user_id'])
user_time_passed = {}

# время начала курса будет в 1 строке (индекс = 0) таблицы 'events', отфильтрованой по пользователю (время совершения первого шага)
# время окончания курса будет в 24 строке (индекс = 23) таблицы 'events_end_time' отфильтрованой по пользователю + 
        # + отфильтрованной по пройденным шагам, за которые дают баллы
events_end_time = events[(events['step_cost'] == 1) & 
            (events['action'] == 'passed')]
    
for user in unique_user_id:
    
    user_df = events[events['user_id'] == user]
    user_df_end_time = events_end_time[events_end_time['user_id'] == user]
    
    # определяем время начала курса
    start_time = user_df.iloc[0].loc['time']
    try:
        # определяем время начала курса (если он окончился)
        end_time = user_df_end_time.iloc[23].loc['time']        
        user_time_passed[user] = end_time - start_time
    except Exception:
        pass
# делаем из словаря таблицу и конвертируем время    
user_time_passed_df = pd.DataFrame(data = list(user_time_passed.values()), index = list(user_time_passed.keys()), columns = ['time'])
user_time_passed_df['time_f'] = pd.to_timedelta(user_time_passed_df['time'], unit='s')

In [197]:
top_10 = user_time_passed_df.sort_values('time').head(10)
top_10

,time,time_f
4313,41390,0 days 11:29:50
1264,207886,2 days 09:44:46
821,348179,4 days 00:42:59
1146,549445,6 days 08:37:25
1124,575883,6 days 15:58:03
2622,847958,9 days 19:32:38
713,868600,10 days 01:16:40
972,1079659,12 days 11:54:19
4280,1081656,12 days 12:27:36
1291,1181486,13 days 16:11:26


In [199]:
# ОТВЕТ:
print('ОТВЕТ:')
list(top_10.index)

ОТВЕТ:


[4313, 1264, 821, 1146, 1124, 2622, 713, 972, 4280, 1291]